# Przykładowy test aplikacji Misty Horizon, przeprowadzony za pomocą Selenium

## Import bibliotek
Importujemy wymagane moduły z Selenium i inne pomocnicze biblioteki.

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


## Klasa `Browser`
Zawiera metody do sterowania przeglądarką, wypełniania formularzy, klikania w przyciski, itd.

In [7]:
class Browser:
    
    browser = None

    def __init__(self): # ANCHOR konstruktor ustawia sterownik przeglądarki na Firefox
        self.browser = webdriver.Firefox()

    def openPage(self, url: str): # ANCHOR funkcja otwierająca stronę z zadanego url
        self.browser.get(url)

        print(f"Page: {url}")

    def closeBrowser(self): # ANCHOR funkcja zamykająca przeglądarkę
        self.browser.close()

        print(f"Browser closed")

    def addInput(self, by: By, value: str, text: str, parent_by = '', parent_val = ''): # ANCHOR funkcja wpisująca dane do pola input
        if parent_by != '' and parent_val != '':                                         
            parent_elem = self.browser.find_element(by=parent_by, value=parent_val)
            field = WebDriverWait(parent_elem, 10).until(EC.element_to_be_clickable((by, value)))
        else:
            field = WebDriverWait(self.browser, 10).until(EC.element_to_be_clickable((by, value)))
        field.send_keys(text)

        print(f"New input data entered: {text}")
        print()

    def clickButton(self, by: By, value: str, parent_by = '', parent_val = ''): # ANCHOR funkcja klikająca w określony przycisk
        if parent_by != '' and parent_val != '':
            parent_elem = self.browser.find_element(by=parent_by, value=parent_val)
            button = WebDriverWait(parent_elem, 10).until(EC.element_to_be_clickable((by, value)))
        else:
            button = WebDriverWait(self.browser, 10).until(EC.element_to_be_clickable((by, value)))
        button.click()

        print(f"Button clicked: {button.text}")
        print()

    def login(self, login, password): # ANCHOR funkcja logująca do aplikacji Misty Horizon
        self.addInput(by=By.TAG_NAME, value='input', text=login)
        self.addInput(by=By.CSS_SELECTOR, value='input[type="password"]', text=password)
        
        self.clickButton(by=By.TAG_NAME, value='button', parent_by=By.TAG_NAME, parent_val='form')

    def testAppMenu(self): # ANCHOR funkcja przechodząca przez menu aplikacji webowej Misty Horizon i testująca działanie funkcjonalności
        side_bar = WebDriverWait(self.browser, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-slot="tabWrapper"]')))
        menu = side_bar.find_element(By.TAG_NAME, 'div')
        menu_buttons = menu.find_elements(By.TAG_NAME, 'button')
        
        for button in menu_buttons:
            button.click()

            side_bar = WebDriverWait(self.browser, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-slot="tabWrapper"]')))
            section_title = side_bar.find_element(By.TAG_NAME, 'h1').text

            time.sleep(1)

            if section_title:
                print()
                print(f"Current section: {section_title}")

            if "statistics" in section_title.lower():
                self.getStatistics()
            elif "intresting spots" in section_title.lower():
                self.getSpots()
            elif "friend requests" in section_title.lower():
                self.getFriendRequests()
            elif "friends" in section_title.lower():
                self.getFriends()

            time.sleep(1)

            print()
            print(f"Selected next button in menu")

    def getStatistics(self): # ANCHOR funkcja pobierająca dane z sekcji statystyk
        side_bar = WebDriverWait(self.browser, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-slot="tabWrapper"]')))
        stat_list = side_bar.find_element(By.TAG_NAME, 'ul')
        stats = stat_list.find_elements(By.TAG_NAME, 'li')

        for i, stat in enumerate(stats):
            print(f"Stat {i+1}: {stat.text}")

    def getSpots(self): # ANCHOR funkcja pobierająca dane z sekcji spotów i przechodząca przez nie
        side_bar = WebDriverWait(self.browser, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-slot="tabWrapper"]')))
        spot_list = side_bar.find_element(By.TAG_NAME, 'ul')
        spots = spot_list.find_elements(By.TAG_NAME, 'button')

        print(f"Total number of spots: {len(spots)}")

        for i, spot in enumerate(spots):
            spot.click()
            
            spot_title = spot.find_element(By.TAG_NAME, 'h3').text
            spot_desc = spot.find_element(By.TAG_NAME, 'span').text
            spot_photo = spot.find_element(By.TAG_NAME, 'img').get_attribute('src')

            print(f"Spot: {i+1}")
            print(f"Spot title:\n{spot_title}")
            print(f"Spot desc:\n{spot_desc}")
            print(f"Spot photo:\n{spot_photo}")
            print()

            time.sleep(5)

    def getFriends(self): # ANCHOR funkcja pobierająca dane z sekcji znajomych
        side_bar = WebDriverWait(self.browser, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-slot="tabWrapper"]')))
        friend_list = side_bar.find_element(By.TAG_NAME, 'ul')
        friends = friend_list.find_elements(By.TAG_NAME, 'li')

        for i, friend in enumerate(friends):
            print(f"Friend {i+1}:\n{friend.text}")

    def getFriendRequests(self): # ANCHOR funkcja pobierająca dane z sekcji zaproszeń do znajomych
        side_bar = WebDriverWait(self.browser, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-slot="tabWrapper"]')))

        if "No friend requests" in side_bar.text:
            print(f"No friend requests!")
        else:
            friend_list = side_bar.find_element(By.TAG_NAME, 'ul')
            friends = friend_list.find_elements(By.TAG_NAME, 'li')

            for i, friend in enumerate(friends):
                print(f"Friend {i+1}:\n{friend.text}")

## Inicjalizacja i testowanie aplikacji Misty Horizon
Tworzymy instancję klasy `Browser`, logujemy się do aplikacji i testujemy menu.

In [9]:
robot = Browser()  # NOTE tworzenie robota przeglądarkowego
robot.openPage('https://mapownik-testbase.web.app/login')  # NOTE otwieranie strony

Page: https://mapownik-testbase.web.app/login


In [10]:
robot.login('tester@horizon.pl', 'tester')

New input data entered: tester@horizon.pl

New input data entered: tester

Button clicked: Sign in



In [11]:
robot.testAppMenu()  # NOTE testowanie funkcjonalności menu


Current section: Statistics
Stat 1: Total area:
350620 m²
Stat 2: Earth %:
0.00000000069%
Stat 3: Total distance:
9.62 km

Selected next button in menu

Current section: Intresting Spots
Total number of spots: 4
Spot: 1
Spot title:
Baza
Spot desc:
kryjówka suchych drani
Spot photo:
https://firebasestorage.googleapis.com/v0/b/mapownik-testbase.firebasestorage.app/o/spots%2FEAuyC3NAvyflfeOZiatkJcLePjm1%2F1735980501.png?alt=media&token=928f666c-b3ab-4966-8918-173970757f43

Spot: 2
Spot title:
Punkt testowy
Spot desc:
nikt nie wie co tu jest
Spot photo:
https://firebasestorage.googleapis.com/v0/b/mapownik-testbase.firebasestorage.app/o/spots%2FEAuyC3NAvyflfeOZiatkJcLePjm1%2F1735980790.png?alt=media&token=329f53cb-6769-4878-85d5-995b4522fcf6

Spot: 3
Spot title:
Zorzowy punk widokowy
Spot desc:
a śnieg sypał i ciągle go było mało
Spot photo:
https://firebasestorage.googleapis.com/v0/b/mapownik-testbase.firebasestorage.app/o/spots%2FEAuyC3NAvyflfeOZiatkJcLePjm1%2F1735998906.png?alt=media&to

In [ ]:
robot.closeBrowser()  # NOTE zamykanie połączenia z przeglądarką

Browser closed
